In [9]:
import json
import pandas as pd
from tqdm.notebook import tqdm

data_file = 'dataset_shingles_maria_sample_5'
l = []

with open(data_file) as reader:
    for obj in tqdm(reader):
        try:
            line = json.loads(obj)
            l.append(line)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue  # Skip invalid lines


df_data = pd.DataFrame(l)

0it [00:00, ?it/s]

Error decoding JSON: Unterminated string starting at: line 1 column 19248 (char 19247)


In [7]:
df_tmp = df_data.iloc[:100]

In [8]:
df_tmp

,num,hard_target,shingles,soft_target
0,150381,False,"[396334907, 1060847325, 1063332450, 1330563559...",False
1,150383,False,"[396334907, 1060847325, 1063332450, 1330563559...",False
2,150384,False,"[227257338, 329292840, 463346217, 1029363358, ...",True
3,150385,False,"[104601096, 422258184, 549984431, 560494154, 5...",True
4,150387,False,"[2039583, 4137730, 5047731, 6762261, 6993977, ...",True
...,...,...,...,...
95,150542,False,"[3269003, 12741328, 14442492, 16473842, 230843...",False
96,150543,False,"[12096343, 19752179, 20324125, 23799634, 25459...",False
97,150544,False,"[409378754, 1014151031, 1437623850, 1850578463...",True
98,150553,False,"[43447116, 55599410, 81608462, 122942364, 1364...",False


In [26]:
from collections import defaultdict

shingle_counts = defaultdict(lambda: {"total": 0, "false_target": 0})

for _, row in df_data.iterrows():
    for shingle in row['shingles']:
        shingle_counts[shingle]["total"] += 1
        if row['hard_target'] or row['soft_target']:  # False target
            shingle_counts[shingle]["false_target"] += 1

bad_shingles = {
    shingle: counts["false_target"] / counts["total"]
    for shingle, counts in shingle_counts.items()
    if counts["false_target"] / counts["total"] > 0.6
}

bad_shingles_df = pd.DataFrame.from_dict(bad_shingles, orient="index", columns=["soft_target"]).reset_index()
bad_shingles_df.rename(columns={"index": "shingle"}, inplace=True)

bad_shingles_df = bad_shingles_df[['shingle']]

bad_shingles_set = set(bad_shingles_df['shingle'])

final_table = []
for _, row in df_tmp.iterrows():
    shingles = row['shingles']

    intersection = len(set(shingles).intersection(bad_shingles_set)) / len(shingles) if shingles else 0

    soft_target = row['soft_target']
    hard_target = row['hard_target']

    final_table.append({
        "num": row['num'],
        "shingles": shingles,
        "soft_target": soft_target,
        "hard_target": hard_target,
        "intersection": intersection
    })

final_df = pd.DataFrame(final_table)



       num                                           shingles  soft_target  \
0   150381  [396334907, 1060847325, 1063332450, 1330563559...        False   
1   150383  [396334907, 1060847325, 1063332450, 1330563559...        False   
2   150384  [227257338, 329292840, 463346217, 1029363358, ...         True   
3   150385  [104601096, 422258184, 549984431, 560494154, 5...         True   
4   150387  [2039583, 4137730, 5047731, 6762261, 6993977, ...         True   
..     ...                                                ...          ...   
95  150542  [3269003, 12741328, 14442492, 16473842, 230843...        False   
96  150543  [12096343, 19752179, 20324125, 23799634, 25459...        False   
97  150544  [409378754, 1014151031, 1437623850, 1850578463...         True   
98  150553  [43447116, 55599410, 81608462, 122942364, 1364...        False   
99  150554  [87343846, 100823462, 191356912, 194972896, 21...        False   

    hard_target  intersection  
0         False      0.000000  

In [27]:
final_df

,num,shingles,soft_target,hard_target,intersection
0,150381,"[396334907, 1060847325, 1063332450, 1330563559...",False,False,0.000000
1,150383,"[396334907, 1060847325, 1063332450, 1330563559...",False,False,0.000000
2,150384,"[227257338, 329292840, 463346217, 1029363358, ...",True,False,1.000000
3,150385,"[104601096, 422258184, 549984431, 560494154, 5...",True,False,1.000000
4,150387,"[2039583, 4137730, 5047731, 6762261, 6993977, ...",True,False,0.999516
...,...,...,...,...,...
95,150542,"[3269003, 12741328, 14442492, 16473842, 230843...",False,False,0.000000
96,150543,"[12096343, 19752179, 20324125, 23799634, 25459...",False,False,0.000000
97,150544,"[409378754, 1014151031, 1437623850, 1850578463...",True,False,1.000000
98,150553,"[43447116, 55599410, 81608462, 122942364, 1364...",False,False,0.000000


In [30]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 23.1 MB/s eta 0:00:00


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import numpy as np

final_df['hard_target_correct'] = np.where(
    (final_df['hard_target'] == False) & (final_df['intersection'] > 0.9), True, final_df['hard_target']
)

final_df['soft_target_correct'] = np.where(
    (final_df['soft_target'] == False) & (final_df['intersection'] > 0.9), True, final_df['soft_target']
)

X = final_df[['intersection']]
y_hard = final_df['hard_target_correct'].astype(int)
y_soft = final_df['soft_target_correct'].astype(int)

X_train_hard, X_test_hard, y_train_hard, y_test_hard = train_test_split(X, y_hard, test_size=0.3, random_state=42)

X_train_soft, X_test_soft, y_train_soft, y_test_soft = train_test_split(X, y_soft, test_size=0.3, random_state=42)

def train_and_evaluate_model(X_train, X_test, y_train, y_test, model_name):
    results = {}

    cat_model = CatBoostClassifier(verbose=0)
    cat_model.fit(X_train, y_train)
    y_pred_cat = cat_model.predict(X_test)
    y_prob_cat = cat_model.predict_proba(X_test)[:, 1]

    results['CatBoost'] = {
        'Accuracy': accuracy_score(y_test, y_pred_cat),
        'ROC-AUC': roc_auc_score(y_test, y_prob_cat),
        'Precision': precision_score(y_test, y_pred_cat),
        'Recall': recall_score(y_test, y_pred_cat),
    }

    xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    xgb_model.fit(X_train, y_train)
    y_pred_xgb = xgb_model.predict(X_test)
    y_prob_xgb = xgb_model.predict_proba(X_test)[:, 1]

    results['XGBoost'] = {
        'Accuracy': accuracy_score(y_test, y_pred_xgb),
        'ROC-AUC': roc_auc_score(y_test, y_prob_xgb),
        'Precision': precision_score(y_test, y_pred_xgb),
        'Recall': recall_score(y_test, y_pred_xgb),
    }

    print(f"\nMetrics for {model_name}:")
    for model, metrics in results.items():
        print(f"{model}:")
        for metric, value in metrics.items():
            print(f"  {metric}: {value:.4f}")

    return results

hard_results = train_and_evaluate_model(X_train_hard, X_test_hard, y_train_hard, y_test_hard, "Hard Target")

soft_results = train_and_evaluate_model(X_train_soft, X_test_soft, y_train_soft, y_test_soft, "Soft Target")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:19:32] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



Metrics for Hard Target:
CatBoost:
  Accuracy: 1.0000
  ROC-AUC: 1.0000
  Precision: 1.0000
  Recall: 1.0000
XGBoost:
  Accuracy: 0.9667
  ROC-AUC: 0.9444
  Precision: 1.0000
  Recall: 0.8889

Metrics for Soft Target:
CatBoost:
  Accuracy: 1.0000
  ROC-AUC: 1.0000
  Precision: 1.0000
  Recall: 1.0000
XGBoost:
  Accuracy: 1.0000
  ROC-AUC: 1.0000
  Precision: 1.0000
  Recall: 1.0000


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:19:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
